In this notebook, we will be using the CNN network for an image classification problem, using CIFAR10 as a dataset.

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images. 


In [15]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as opt
import torch.utils.data as tud
import torchvision
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Defining transformation
Data augmentation

In [22]:
from torchvision import transforms
transTrain = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=(0.5, 0.5, 0.5),std=(0.5, 0.5, 0.5)),
                                transforms.RandomHorizontalFlip(),transforms.RandomRotation(20) ])
transTest = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=(0.5, 0.5, 0.5),std=(0.5, 0.5, 0.5))])

# Datasets
Loading the CIFAR10 dataset with transformations which are defined above
seperating the train dataset into train and validation.

In [23]:
from torchvision import datasets
from torch.utils.data.sampler import SubsetRandomSampler

batchSize = 50  #the number of samples in a batch for training
valPer = 0.1    #Validation percentage

trainData = datasets.CIFAR10('data/train',download=True,train=True,transform=transTrain)  # Creating Train dataset
testData = datasets.CIFAR10('data/test',download=True,train=False,transform=transTrain)   # Creating Test dataset


trainSize = len(trainData)
idxs = list(range(trainSize))
np.random.shuffle(idxs)
spl = int(np.floor(valPer * trainSize))          #split the dataset to train and validation sets
trainIdxs, valIdxs = idxs[spl:], idxs[:spl]

trainSamp = SubsetRandomSampler(trainIdxs)
valSamp = SubsetRandomSampler(valIdxs)

trainLoader = tud.DataLoader(trainData,batch_size=batchSize,sampler=trainSamp,num_workers=0)
valLoader = tud.DataLoader(trainData,batch_size=batchSize,sampler=valSamp,num_workers=0)
testLoader = tud.DataLoader(testData,batch_size=batchSize,num_workers=0,shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


# Defining the CNN architecture:
 
*   3 convolutional layer
*   relu function 
*   maxpooling after each convolutional layer
*   dropout to avoid overfitting 

In [24]:
#Defining the cnn network
class CNN(nn.Module):
  def __init__(self):
      super(CNN,self).__init__()
      self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
      self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
      self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
      self.fc1 = nn.Linear(64 * 4 * 4, 500)
      self.fc2 = nn.Linear(500, 10)
      self.maxPool = nn.MaxPool2d(kernel_size = 2, stride = 2)
      self.dropout = nn.Dropout(0.25)
      self.relu = nn.ReLU()

  def forward(self,x):
      out = self.maxPool(func.relu(self.conv1(x)))
      out = self.maxPool(func.relu(self.conv2(out)))
      out = self.maxPool(func.relu(self.conv3(out)))
      out = out.view(-1, 64 * 4 * 4)
      out = self.dropout(out)
      out = func.relu(self.fc1(out))
      out = self.dropout(out)
      out = self.fc2(out)
      return out

# Train the CNN Network:
The error on train and validation sets are decreased over time. we save the network with minimum loss and use it later in prediction.

In [25]:
#Training the model
import torch.optim as optim

cnnNet = CNN()
cnnNet.to(device)
criterion = nn.CrossEntropyLoss()   # Settin Loss function with criterion
lr = 0.01 #learning rate
optimizer = optim.SGD(cnnNet.parameters(), lr=lr, momentum = 0.9)

nepc = 20 # number of epochs


desiredLoss = 0.001
minLoss = np.Inf

for epc in range(1,nepc+1):
    trainLoss = 0
    cnnNet.train()
    #-----------Training
    for images, labels in trainLoader: 
      images = images.to(device)
      labels = labels.to(device)
      optimizer.zero_grad()
      pred = cnnNet(images)
      loss = criterion(pred, labels)
      loss.backward()
      optimizer.step()
      trainLoss += loss.item()
    trainLoss = trainLoss/len(trainLoader)
    print('Epoch no {} , Loss: {:.4f}'.format(epc, trainLoss))
   
    #-----------Validation
    cnnNet.eval()
    valLoss = 0
    valLoss2 = 0
    for images,labels in valLoader:
      images = images.to(device)
      labels = labels.to(device)
      pred = cnnNet(images)
      loss = criterion(pred, labels)
      valLoss += loss.item()

    valLoss = valLoss/len(valLoader)
    if valLoss < minLoss:  #saving the model with minimum loss
      torch.save(cnnNet.state_dict(), 'cnnNet')
      minLoss = valLoss
      print('Validation loss: {:.4f}'.format(valLoss))
    if valLoss < desiredLoss:
      break


Epoch no 1 , Loss: 1.7765
Validation loss: 1.4290
Epoch no 2 , Loss: 1.3638
Validation loss: 1.2198
Epoch no 3 , Loss: 1.2202
Validation loss: 1.0970
Epoch no 4 , Loss: 1.1256
Validation loss: 1.0324
Epoch no 5 , Loss: 1.0645
Validation loss: 0.9647
Epoch no 6 , Loss: 1.0078
Validation loss: 0.8845
Epoch no 7 , Loss: 0.9689
Validation loss: 0.8696
Epoch no 8 , Loss: 0.9372
Validation loss: 0.8513
Epoch no 9 , Loss: 0.9132
Validation loss: 0.8165
Epoch no 10 , Loss: 0.8884
Epoch no 11 , Loss: 0.8791
Validation loss: 0.7915
Epoch no 12 , Loss: 0.8537
Epoch no 13 , Loss: 0.8365
Validation loss: 0.7843
Epoch no 14 , Loss: 0.8197
Validation loss: 0.7803
Epoch no 15 , Loss: 0.8094
Validation loss: 0.7682
Epoch no 16 , Loss: 0.7991
Epoch no 17 , Loss: 0.7830
Epoch no 18 , Loss: 0.7850
Validation loss: 0.7314
Epoch no 19 , Loss: 0.7705
Validation loss: 0.7222
Epoch no 20 , Loss: 0.7651


In [26]:
cnnNet.load_state_dict(torch.load('cnnNet'))  #Loading the best model, with minimum error

<All keys matched successfully>

In [29]:
testLoss = 0
correct = 0
cnnNet.eval()
total = 0
for images,labels in testLoader:
    images = images.to(device)
    labels = labels.to(device)
    pred = cnnNet(images)
    loss = criterion(pred, labels)
    testLoss += loss.item()*images.size(0)
    total += labels.size(0)
    _, pred = torch.max(pred, 1)
    correct += (pred == labels).sum().item()
testLoss = testLoss/len(testLoader)

print('The final accuracy of the network on test images: {:.4f}'.format(correct/total*100))


The final accuracy of the network on test images: 74.3300
